In [ ]:
import geopandas as gpd
import rioxarray as rxr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pandas as pd
import seaborn as sns

from src import constants
from src.utils import raster_utils

# 0. Read in and overlay disturbance + burn data

In [ ]:
degrade_pts = gpd.read_feather(
    constants.RESULTS_PATH / "glad_treatment_0d.feather"
)
rst = rxr.open_rasterio(
    constants.DATA_PATH / "Disturb_type" / "fire" / "modis_burndate_2019-2022_amazon.tif"
)
# rst = rxr.open_rasterio(
#     constants.DATA_PATH / "annual_burndate" / "annual_burndate.tif"
# )

In [ ]:
xs = degrade_pts.t2_geom.x.values
ys = degrade_pts.t2_geom.y.values

rst_x_idxs = raster_utils.get_idx(rst.x.data, xs)
rst_y_idxs = raster_utils.get_idx(rst.y.data, ys)

In [ ]:
yrs = [2019, 2020, 2021, 2022]
degrade_pts["t1_yyyydoy"] = (degrade_pts["t1_absolute_time"].dt.year) * 1000 + degrade_pts["t1_absolute_time"].dt.dayofyear
degrade_pts["t2_yyyydoy"] = (degrade_pts["t2_absolute_time"].dt.year) * 1000 + degrade_pts["t2_absolute_time"].dt.dayofyear

for i, y in enumerate(yrs):
    day = rst.data[i, rst_y_idxs, rst_x_idxs]
    degrade_pts[f"burn_bool_{y}"] = np.logical_and(
        day != 0, np.logical_and(
        day + (y * 1000) > degrade_pts["t1_yyyydoy"],
        day + (y * 1000) < degrade_pts["t2_yyyydoy"],
        ))

degrade_pts["interceding_burn"] = degrade_pts[[f"burn_bool_{y}" for y in yrs]].any(axis=1)

### 0.1 Bucket PAI according to vertical thirds of pre-disturbance height

In [ ]:
print(len(degrade_pts[degrade_pts["interceding_burn"]]))
with pd.option_context("mode.chained_assignment", None):
    # Set a cutoff point for "upper" and "lower" PAI for each sample:
    # one third of the pre-disturbance height, rounded up to the nearest 5
    # (PAI buckets are 5 units wide)
    degrade_pts["t1_mid_tch_bucket"] = degrade_pts.t1_rh_98_a0.apply(
        lambda x: int((x / 3) // 5 * 5)
    )
    degrade_pts["t1_upper_pai"] = np.nan
    degrade_pts["t2_upper_pai"] = np.nan
    degrade_pts["t1_lower_pai"] = np.nan
    degrade_pts["t2_lower_pai"] = np.nan
    # For each group of samples with the same cutoff point,
    # sum the PAI values above and below the cutoff point
    for el in degrade_pts.t1_mid_tch_bucket.unique():
        idx = degrade_pts.t1_mid_tch_bucket == el
        print(f"{el}: {idx.sum()}")
        degrade_pts["t1_upper_pai"][idx] = (
            degrade_pts[f"t1_pai_{el}"][idx] - degrade_pts.t1_pai_100[idx]
        )
        degrade_pts["t2_upper_pai"][idx] = (
            degrade_pts[f"t2_pai_{el}"][idx] - degrade_pts.t2_pai_100[idx]
        )

        degrade_pts["t1_lower_pai"][idx] = (
            degrade_pts.t1_pai_0[idx] - degrade_pts[f"t1_pai_{el}"][idx]
        )
        degrade_pts["t2_lower_pai"][idx] = (
            degrade_pts.t2_pai_0[idx] - degrade_pts[f"t2_pai_{el}"][idx]
        )

# 1. Plotting

In [ ]:
import matplotlib

textsize = 15
ticksize = 13
plt.rc("xtick", labelsize=ticksize)
plt.rc("ytick", labelsize=ticksize)
red = matplotlib.cm.get_cmap("coolwarm")(1.0)
blue = matplotlib.cm.get_cmap("coolwarm")(0.0)
red1 = "tab:red"
blue1 = "tab:blue"
burned = degrade_pts[degrade_pts["interceding_burn"]]
cond = burned.t2_rh_98_a0 >= (0.65 * burned.t1_rh_98_a0)
with pd.option_context("mode.chained_assignment", None):
    burned["canopy_lost"] = True
    burned.loc[cond, "canopy_lost"] = False
    burned = burned[burned.t1_rh_98_a0 >= 5]
    burned = burned[burned.t1_rh_50_a0 >= 2.5]


fig = plt.figure(layout="constrained", figsize=(15, 10))
subfigs = fig.subfigures(2, 1, hspace=0.15, height_ratios=[1, 1])
axs_top = subfigs[0].subplots(1, 3)
axs_bottom = subfigs[1].subplots(1, 2)

axi = axs_top[0]
axi.scatter(
    burned.t1_rh_98_a0,
    burned.t2_rh_98_a0,
    s=2,
    c=burned.canopy_lost,
    cmap="coolwarm",
)
axi.set_title("RH 98", fontsize=textsize)
maxh = 55
axi.set_xlim(0, maxh)
axi.set_ylim(2, maxh)
axi.plot([0, maxh], [0, maxh], color="gray", linestyle="dashed")

axi = axs_top[1]
axi.scatter(
    burned[burned.canopy_lost == True].t1_rh_50_a0,
    burned[burned.canopy_lost == True].t2_rh_50_a0,
    s=2,
    color=red,
)
axi.scatter(
    burned[burned.canopy_lost == False].t1_rh_50_a0,
    burned[burned.canopy_lost == False].t2_rh_50_a0,
    s=2,
    color=blue,
)
axi.set_title("RH 50", fontsize=textsize)
maxh = 20
axi.set_xlim(0, maxh)
axi.set_ylim(-0.5, maxh)
axi.plot([0, maxh], [0, maxh], color="gray", linestyle="dashed")

axi = axs_top[2]
axi.scatter(
    burned[burned.canopy_lost == True].t1_agbd_a0,
    burned[burned.canopy_lost == True].t2_agbd_a0,
    s=2,
    color=red,
)
axi.scatter(
    burned[burned.canopy_lost == False].t1_agbd_a0,
    burned[burned.canopy_lost == False].t2_agbd_a0,
    s=2,
    color=blue,
)
axi.set_title("AGBD", fontsize=textsize)
maxh = 300
axi.set_xlim(0, maxh)
axi.set_ylim(-1.5, maxh)
axi.plot([0, maxh], [0, maxh], color="gray", linestyle="dashed")
subfigs[0].supxlabel("Time 1", fontsize=textsize)
subfigs[0].supylabel("Time 2", fontsize=textsize)


sns.set_theme(style="ticks")
axi = axs_bottom[0]
sns.kdeplot(x=burned.t1_upper_pai, ax=axi, color="tab:green", fill=True, )
sns.kdeplot( x=burned.t2_upper_pai[burned.canopy_lost == False], ax=axi, color=blue1, fill=True, )
sns.kdeplot( x=burned.t2_upper_pai[burned.canopy_lost == True], ax=axi, color=red1, fill=True, )
axi.set_title("Upper & mid stories", fontsize=textsize)
axi.set_xlabel(None)
axi.set_ylabel(None)

axi = axs_bottom[1]
sns.kdeplot(x=burned.t1_lower_pai, ax=axi, color="tab:green", fill=True, )
sns.kdeplot( x=burned.t2_lower_pai[burned.canopy_lost == False], ax=axi, color=blue1, fill=True, )
sns.kdeplot( x=burned.t2_lower_pai[burned.canopy_lost == True], ax=axi, color=red1, fill=True, )
axi.set_title("Understory", fontsize=textsize)
axi.set_xlabel(None)
axi.set_ylabel(None)

subfigs[1].supxlabel("PAI", fontsize=textsize)
subfigs[1].supylabel("Density", fontsize=textsize)

legend1_elements = [
    Line2D( [0], [0], marker="o", color=blue, label="Canopy preserved", markerfacecolor=blue, markersize=7, linestyle="None",),
    Line2D( [0], [0], marker="o", color=red, label="Canopy lost", markerfacecolor=red, markersize=7, linestyle="None",), ]
subfigs[0].legend(
    handles=legend1_elements, loc="lower center", bbox_to_anchor=(0.5, -0.1), fancybox=False, shadow=False, ncol=2, fontsize=textsize,
)

legend2_elements = [
    Line2D( [0], [0], marker="o", color="tab:green", label="Pre-disturbance", markerfacecolor="tab:green", markersize=7, linestyle="None",),
    Line2D( [0], [0], marker="o", color=blue1, label="Post-disturbance,\n canopy preserved", markerfacecolor=blue1, markersize=7, linestyle="None",),
    Line2D( [0], [0], marker="o", color=red1, label="Post-disturbance,\n canopy lost", markerfacecolor=red1, markersize=7, linestyle="None",),
    ]
subfigs[1].legend(
    handles=legend2_elements, loc="upper center", bbox_to_anchor=(0.5, 0), fancybox=False, shadow=False, ncol=3, fontsize=textsize,
)
# fig.suptitle("Structural impacts of burning")

In [ ]:
print(f"Total samples meeting conditions: {len(burned)}")

# Percent of [burned forest] in which the canopy was preserved
tot = len(burned)
sub = len(burned[cond])
print(f"Percent of burned forest overall, in which the canopy was preserved: {sub/tot * 100:.1f}%")

# Percent of [burned samples when the canopy was preserved] in which the understory was lost
tot = len(burned[cond])
sub = len(burned[cond & (burned.t2_rh_50_a0 <= 2.5)])
print(f"Percent of blue points in which the understory was lost: {sub/tot * 100:.1f}%")

# Percent of [burned forest] in which the canopy was preserved but the understory was lost
tot = len(burned)
sub = len(burned[cond & (burned.t2_rh_50_a0 <= 2.5)])
print(f"Percent of burned forest overall, in which the canopy was preserved but the understory lost: {sub/tot * 100:.1f}%")